Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in dow30.csv
dow30 = pd.read_csv('dow30.csv')

# Remove the exchange from the beginning of the symbol.
def _symbol(row):
    return row['Symbol'].split(':')[-1].strip()

dow30['Symbol'] = dow30.apply(_symbol, axis=1)
dow30.drop(columns=['Exchange', 'Date added', 'Notes', 'Index weighting'], inplace=True)
dow30.rename(columns={'Symbol': 'Symbol',
                      'Company':'Description',
                      'Industry':'Asset Class'}, inplace=True)

dow30.set_index("Symbol", inplace=True)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology
KO,Coca-Cola,Drink industry


In [6]:
# Read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Conglomerate': 'US Stocks:Industrials',
 'Financial services': 'US Stocks:Financials',
 'Pharmaceutical industry': 'US Stocks:Healthcare',
 'Biopharmaceutical': 'US Stocks:Healthcare',
 'Information technology': 'US Stocks:Technology',
 'Aerospace and defense': 'US Stocks:Industrials',
 'Construction and mining': 'US Stocks:Industrials',
 'Petroleum industry': 'US Stocks:Energy',
 'Food industry': 'US Stocks:Consumer Staples',
 'Drink industry': 'US Stocks:Consumer Staples',
 'Chemical industry': 'US Stocks:Materials',
 'Retailing': 'US Stocks:Consumer Discretionary',
 'Apparel': 'US Stocks:Consumer Discretionary',
 'Fast-moving consumer goods': 'US Stocks:Consumer Staples',
 'Managed health care': 'US Stocks:Healthcare',
 'Telecommunication': 'US Stocks:Communication Services',
 'Broadcasting and entertainment': 'US Stocks:Communication Services',
 'Home Improvement': 'US Stocks:Consumer Discretionary',
 'Semiconductor industry': 'US Stocks:Technology',
 'Clothing industry': 'US Sto

In [7]:
# Map dow30 GICS sectors to brownbear defined asset classes.
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

dow30['Asset Class'] = dow30.apply(_asset_class, axis=1)

# Yahoo finance uses '-' where '.' is used in symbol names.
dow30.index = dow30.index.str.replace('.', '-', regex=False)
dow30

,Description,Asset Class
Symbol,,
MMM,3M,US Stocks:Industrials
AXP,American Express,US Stocks:Financials
AMGN,Amgen,US Stocks:Healthcare
AAPL,Apple,US Stocks:Technology
BA,Boeing,US Stocks:Industrials
CAT,Caterpillar,US Stocks:Industrials
CVX,Chevron,US Stocks:Energy
CSCO,Cisco,US Stocks:Technology
KO,Coca-Cola,US Stocks:Consumer Staples


In [8]:
# Make symbols list.
symbols = list(dow30.index)
#symbols

In [9]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

MMM AXP AMGN AAPL BA CAT CVX CSCO KO DIS DOW GS HD HON IBM INTC JNJ JPM MCD MRK MSFT NKE PG CRM TRV UNH VZ V WBA WMT 


In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DIS,...,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,121.18,81.28,123.86,24.47,113.66,71.50,76.63,20.99,31.60,87.09,...,40.45,43.08,69.98,59.24,85.65,87.92,30.30,62.25,56.04,71.17
2015-01-05,118.44,79.13,122.39,23.78,112.87,67.72,73.57,20.57,31.60,85.82,...,40.08,42.39,69.64,58.17,84.62,86.47,30.04,60.87,54.93,70.96
2015-01-06,117.18,77.45,118.44,23.78,111.54,67.29,73.53,20.56,31.84,85.36,...,39.49,42.14,69.33,57.20,83.86,86.29,30.35,60.48,55.07,71.51
2015-01-07,118.03,79.14,122.58,24.11,113.27,68.33,73.47,20.75,32.24,86.24,...,39.99,43.01,69.69,56.93,85.29,87.17,30.15,61.29,56.48,73.40
2015-01-08,120.86,80.26,122.14,25.04,115.28,69.03,75.15,20.91,32.63,87.13,...,41.17,44.00,70.49,58.59,87.07,91.34,30.80,62.11,57.18,74.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,106.33,185.57,284.16,193.60,260.44,290.36,151.05,50.09,58.32,91.02,...,374.58,108.04,145.28,266.34,185.80,520.31,37.49,258.43,26.22,156.65
2023-12-26,108.11,185.96,283.90,193.05,262.79,295.63,152.41,50.28,58.56,90.95,...,374.66,108.02,145.94,266.22,187.09,520.03,37.48,259.16,26.61,156.41
2023-12-27,108.74,187.06,286.53,193.15,262.10,298.12,151.91,50.44,58.71,90.38,...,374.07,107.13,146.06,266.72,188.62,522.79,37.31,258.93,26.64,157.88


In [11]:
# Sample symbol.
symbol = 'MMM'

In [12]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns[symbol]

-1.99598963573151

In [13]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [14]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.21916621882028162

In [15]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola[symbol]

0.10241509703980027

In [16]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   119.88
2015-02-28   125.34
2015-03-31   122.59
2015-04-30   116.23
2015-05-31   118.98
2015-06-30   115.40
2015-07-31   113.19
2015-08-31   107.05
2015-09-30   106.77
2015-10-31   118.40
2015-11-30   118.70
2015-12-31   114.19
2016-01-31   114.47
2016-02-29   119.77
2016-03-31   127.23
2016-04-30   127.80
2016-05-31   129.37
2016-06-30   134.60
2016-07-31   137.09
2016-08-31   138.62
2016-09-30   136.30
2016-10-31   127.84
2016-11-30   133.68
2016-12-31   138.99
2017-01-31   136.07
2017-02-28   145.99
2017-03-31   149.89
2017-04-30   153.42
2017-05-31   161.14
2017-06-30   164.08
2017-07-31   158.54
2017-08-31   161.95
2017-09-30   166.38
2017-10-31   182.46
2017-11-30   193.70
2017-12-31   187.51
2018-01-31   199.56
2018-02-28   188.72
2018-03-31   175.91
2018-04-30   155.77
2018-05-31   159.13
2018-06-30   158.72
2018-07-31   171.30
2018-08-31   171.31
2018-09-30   171.14
2018-10-31   154.53
2018-11-30   170.03
2018-12-31   155.82
2019-01-31   163.80
2019-02-28   17

In [17]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2023-08-31   -0.03
2023-09-30   -0.12
2023-10-31   -0.03
2023-11-30    0.11
2023-12-31    0.10
Freq: M, Name: MMM, Length: 108, dtype: float64

In [18]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.2538905958439827

In [19]:
# Read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: DOW 30 investment options. 10/02/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [20]:
# For each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = lines.copy()

for i, (index, row) in enumerate(dow30.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [21]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')